https://www.science.org/doi/10.1126/sciadv.abh0146

Chaotic neural dynamics facilitate probabilistic computations through sampling

Effective Learning with Node Perturbation in Multi-Layer Neural Networks (fig1は図の参考になる．)
On the stability and scalability of node perturbation learning
Node perturbation learning without noiseless baseline

どれも効率は良くない．

### node perturbation (NP)

$$
\mathbf{z}_{\ell+1}=f_\ell\left(\mathbf{W}_\ell \mathbf{z}_\ell +\mathbf{b}_\ell\right)
$$

$$
\tilde{\mathbf{z}}_{\ell+1}=f_\ell\left(\mathbf{W}_\ell \tilde{\mathbf{z}}_\ell +\mathbf{b}_\ell+\xi_\ell \right)
$$

$$
\delta \mathcal{L}=\mathcal{L}(\tilde{\mathbf{z}}_{L})-\mathcal{L}(\mathbf{z}_{L})
$$

$$
\Delta W_\ell =- \delta \mathcal{L} \frac{\xi_\ell}{\sigma^2} \mathbf{z}_{\ell}
$$

### Weight perturbation (WP)
https://journals.aps.org/prx/abstract/10.1103/PhysRevX.13.021006
Weight Perturbation Learning Performs Similarly or Better than Node Perturbation on Broad Classes of Temporally Extended Tasks

$$
\mathbf{z}_{\ell+1}=f_\ell\left(\mathbf{W}_\ell \mathbf{z}_\ell +\mathbf{b}_\ell\right)
$$

$$
\tilde{\mathbf{z}}_{\ell+1}=f_\ell\left((\mathbf{W}_\ell+\xi_\ell) \tilde{\mathbf{z}}_\ell +\mathbf{b}_\ell \right)
$$

$$
\delta \mathcal{L}=\mathcal{L}(\tilde{\mathbf{z}}_{L})-\mathcal{L}(\mathbf{z}_{L})
$$

$$
\Delta W_\ell = -\delta \mathcal{L} \frac{\xi_\ell}{\sigma^2}
$$

### Directional Gradient Descent

Can Forward Gradient Match Backpropagation?
https://proceedings.mlr.press/v202/fournier23a.html

https://oumpy.github.io/blog/2022/02/directional_gradient_optimization.html
この記事では(Silver, et al., "Learning by Directional Gradient Descent." ICLR. 2021)および(Baydin, et al., "Gradients without Backpropagation", arXiv, 2022)の解説&実装を行います．

両者とも摂動 (perturbation) と方向微分(directional gradient) を用いて勾配を近似することで，誤差逆伝播法 (backpropagation)を用いずにニューラルネットワークを訓練するという手法を提案しています．gradient-free optimizationの一種とも言えるでしょう．以後，Silverらの提案手法をDODGE(Deep Online Directional Gradient Estimate), Baydinらの提案手法をFGD (Forward gradient descent)と呼ぶことにしま

$$
\begin{align*}
&\textbf{if}\ \text{DODGE:}\\
&\quad \mathbf{v} \sim \{-1, 1\}^p\\
&\textbf{else if}\ \text{FGD:}\\
&\quad \mathbf{v} \sim \mathcal{N}(0, \mathbf{I})\\
&g(\boldsymbol{\theta}, \mathbf{x}) = (\nabla L(\boldsymbol{\theta}, \mathbf{x})\cdot \mathbf{v})\cdot \mathbf{v}\\
&\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta \cdot g(\boldsymbol{\theta}, \mathbf{x})
\end{align*}
$$す．